![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# United Nations



### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
import pandas as pd
import plotly_express as px
import folium
import geopandas as gpd
import plotly.graph_objs as go
import ipywidgets
from ipywidgets import interact
print("Libraries imported.")

In [ ]:
HDI_components = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/UnitedNations/HDI_components.xlsx")
HDI_inequality = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/UnitedNations/HDI_inequality_adjusted.xlsx")
file_path = "https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/UnitedNations/HDI_multidimensional_poverty.xlsx"
sheet1_name = 'table1'
sheet2_name = 'table2'
HDI_multidimensional_poverty = pd.read_excel(file_path, sheet_name=sheet1_name)
HDI_multidimensional_poverty2 = pd.read_excel(file_path, sheet_name=sheet2_name)
HDI_years = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/united-nations/HDI_years.xlsx")

display(HDI_components)
display(HDI_inequality)
display(HDI_multidimensional_poverty)
display(HDI_multidimensional_poverty2)
display(HDI_years)

### Examining HDI Trends

In [ ]:
HDI_years

In [ ]:
HDI_cols = HDI_years.columns[2:10]
for column in HDI_cols:
    HDI_years[column] = pd.to_numeric(HDI_years[column], errors='coerce')
    print(f"Maximum value in {column}: {HDI_years[column].max()}")
    max_hdi_country = HDI_years.loc[HDI_years[column].idxmax()]['Country']
    print(f"Country with the highest HDI: {max_hdi_country}")

In [ ]:
def update_scatter_plot(selected_year):
    data = []
    for country in HDI_years['Country']:
        trace = go.Scatter(x=[country], y=[HDI_years.loc[HDI_years['Country'] == country, selected_year].values[0]], mode='markers', name=country)
        data.append(trace)

    layout = go.Layout(title=f'HDI Scatter Plot ({selected_year})', xaxis=dict(showticklabels=False, title='Country'), yaxis=dict(title='HDI'))
    selected_year_fig = go.Figure(data=data, layout=layout)
    selected_year_fig.show()

interact(update_scatter_plot, selected_year=HDI_cols)

In [ ]:
countries_geojson = gpd.read_file('https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/UnitedNations/geopandas.geojson')
countries_geojson

In [ ]:
geojson_country_names = countries_geojson['ADMIN']
hdi_country_names = HDI_years['Country']
matching_countries = set(geojson_country_names).intersection(hdi_country_names)
print(matching_countries)

In [ ]:
geojson_country_names = countries_geojson['ADMIN']

hdi_country_names = HDI_years['Country']

matching_countries = set(geojson_country_names).intersection(hdi_country_names)
non_matching_countries = set(hdi_country_names) - matching_countries
non_matching_countries_geojson = set(geojson_country_names) - matching_countries

print(f'Non-matching geojson: {non_matching_countries_geojson}')
print(f'Non-matching dataframe countries: {non_matching_countries}')

In [ ]:
no_HDI_years = HDI_years[HDI_years['HDI rank'].isnull()]
display(no_HDI_years)
HDI_years.dropna(subset=['HDI rank'], inplace=True)

In [ ]:
mapping = {
    'Russian Federation': 'Russia',
    'Micronesia (Federated States of)': 'Federated States of Micronesia',
    'Cabo Verde': 'Cape Verde',
    "Korea (Democratic People's Rep. of)": 'North Korea',
    'North Macedonia': 'Macedonia',
    'Bahamas': 'The Bahamas',
    'Tanzania (United Republic of)': 'United Republic of Tanzania',
    'Türkiye': 'Turkey',
    'Serbia': 'Republic of Serbia',
    'Eswatini (Kingdom of)': 'Swaziland',
    'Guinea-Bissau': 'Guinea Bissau',
    'Timor-Leste': 'East Timor',
    "Lao People's Democratic Republic": 'Laos',
    'Congo': 'Republic of Congo',
    'Syrian Arab Republic': 'Syria',
    'Brunei Darussalam': 'Brunei',
    'Viet Nam': 'Vietnam',
    'Iran (Islamic Republic of)': 'Iran',
    'Czechia': 'Czech Republic',
    'Congo (Democratic Republic of the)': 'Democratic Republic of the Congo',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Moldova (Republic of)': 'Moldova',
    'Korea (Republic of)': 'South Korea',
    "Côte d'Ivoire": 'Ivory Coast',
    'Palestine, State of': 'Palestine',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Hong Kong, China (SAR)': 'Hong Kong S.A.R.',
    'United States': 'United States of America'
}

HDI_years['Country'] = HDI_years['Country'].replace(mapping)

In [ ]:
merged_data = pd.merge(HDI_years, countries_geojson, left_on='Country', right_on='ADMIN', how='left')
merged_data

In [ ]:
HDI_by_country = ipywidgets.Output(layout={'border': '1px solid black'})

column_names = merged_data.columns[2:10].tolist()
dropdown_options = ipywidgets.Dropdown(
    options=column_names,
    value=column_names[0],
    description='Column:',
    disabled=False
)

def update_choropleth(change):
    HDI_by_country.clear_output()
    with HDI_by_country:
        m = folium.Map()
        folium.Choropleth(
            geo_data=countries_geojson,
            data=merged_data,
            columns=['ADMIN', dropdown_options.value],  
            key_on='feature.properties.ADMIN',  
            fill_color='YlGn',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f'{dropdown_options.value} by Province',
        ).add_to(m)
        display(m)

dropdown_options.observe(update_choropleth, names='value')
display(dropdown_options)
update_choropleth({'new': column_names[0]})

HDI_by_country

In [ ]:
HDI_components

In [ ]:
countries = HDI_components.Country.unique()
for country in countries:
    print(country)

In [ ]:
no_HDI = HDI_components[HDI_components['HDI rank'].isnull()]
no_HDI